In [8]:
import os
import cv2
import numpy as np
import mediapipe as mp
from keras.utils import np_utils
from keras.models import Sequential
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [2]:
mp_face_detection = mp.solutions.face_detection

face_detector = mp_face_detection.FaceDetection(
    min_detection_confidence=0.6
)
valid_split_path = '../validation/raw'
valid_cropped_split_path = '../validation/cropped'

In [3]:
model = Sequential()

model.add(Conv2D(64, (3,3), activation='relu', padding='same', kernel_regularizer=l2(1e-7), kernel_initializer='he_uniform', name='block1_conv1', input_shape=(48, 48, 1) ))
model.add(Conv2D(64, (3,3), activation='relu', padding='same', kernel_regularizer=l2(1e-7), kernel_initializer='he_uniform', name='block1_conv2'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), name='block1_maxpool'))

model.add(Conv2D(128, (3,3), activation='relu', padding='same', kernel_regularizer=l2(1e-7), kernel_initializer='he_uniform', name='block2_conv1'))
model.add(Conv2D(128, (3,3), activation='relu', padding='same', kernel_regularizer=l2(1e-7), kernel_initializer='he_uniform', name='block2_conv2'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), name='block2_maxpool'))

model.add(Conv2D(256, (3,3), activation='relu', padding='same', kernel_regularizer=l2(1e-7), kernel_initializer='he_uniform', name='block3_conv1'))
model.add(Conv2D(256, (3,3), activation='relu', padding='same', kernel_regularizer=l2(1e-7), kernel_initializer='he_uniform', name='block3_conv2'))
model.add(Conv2D(256, (3,3), activation='relu', padding='same', kernel_regularizer=l2(1e-7), kernel_initializer='he_uniform', name='block3_conv3'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), name='block3_maxpool'))

model.add(Conv2D(512, (3,3), activation='relu', padding='same', kernel_regularizer=l2(1e-7), kernel_initializer='he_uniform', name='block4_conv1'))
model.add(Conv2D(512, (3,3), activation='relu', padding='same', kernel_regularizer=l2(1e-7), kernel_initializer='he_uniform', name='block4_conv2'))
model.add(Conv2D(512, (3,3), activation='relu', padding='same', kernel_regularizer=l2(1e-7), kernel_initializer='he_uniform', name='block4_conv3'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), name='block4_maxpool'))

model.add(Conv2D(512, (3,3), activation='relu', padding='same', kernel_regularizer=l2(1e-7), kernel_initializer='he_uniform', name='block5_conv1'))
model.add(Conv2D(512, (3,3), activation='relu', padding='same', kernel_regularizer=l2(1e-7), kernel_initializer='he_uniform', name='block5_conv2'))
model.add(Conv2D(512, (3,3), activation='relu', padding='same', kernel_regularizer=l2(1e-7), kernel_initializer='he_uniform', name='block5_conv3'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), name='block5_maxpool'))

model.add(Flatten())
model.add(Dense(4096, kernel_regularizer=l2(1e-7), activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(4096, kernel_regularizer=l2(1e-7), activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(8, activation='softmax'))

model.compile(optimizer=Adam(learning_rate=0.0002), loss='categorical_crossentropy', metrics=['accuracy']) 
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 48, 48, 64)        640       
                                                                 
 block1_conv2 (Conv2D)       (None, 48, 48, 64)        36928     
                                                                 
 block1_maxpool (MaxPooling2  (None, 24, 24, 64)       0         
 D)                                                              
                                                                 
 block2_conv1 (Conv2D)       (None, 24, 24, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 24, 24, 128)       147584    
                                                                 
 block2_maxpool (MaxPooling2  (None, 12, 12, 128)      0         
 D)                                                     

In [10]:
model.load_weights('../models/VGG16-best_loss_model.h5')
class_labels = {0: 'angry', 1: 'contempt', 2: 'disgust', 3: 'fear', 4: 'happy', 5: 'neutral', 6: 'sad', 7: 'surprise'}
label_classes = {'angry': 0, 'contempt': 1, 'disgust': 2, 'fear': 3, 'happy': 4, 'neutral': 5, 'sad': 6, 'surprise': 7}

In [5]:
sub_dirs = os.listdir(valid_split_path)
corrupted = 0

for sd in sub_dirs:
    sd_path = os.path.join(valid_split_path, sd)
    img_files = os.listdir(sd_path)

    for i, f in enumerate(img_files):
        img_file_path = os.path.join(sd_path, f)

        try:
            img = cv2.imread(img_file_path)
            h, w, _ = img.shape
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            face_detect_results = face_detector.process(img_rgb)
            img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
            
            if face_detect_results.detections:
                for face in face_detect_results.detections:
                    rel_bound_rect = face.location_data.relative_bounding_box
                    face_x, face_w, face_y, face_h = int(rel_bound_rect.xmin * w), int(rel_bound_rect.width * w), int(rel_bound_rect.ymin * h), int(rel_bound_rect.height * h)
                    cropped_face = img_gray[face_y: face_y + face_h, face_x: face_x + face_w]
                    processed_img = cv2.resize(cropped_face, (48, 48))
                    rescaled = processed_img / 255.0
                    reshaped = np.reshape(rescaled, (1, rescaled.shape[0], rescaled.shape[1], 1))
                    preds = model.predict(reshaped)
                    pred = np.argmax(preds, axis=1)[0]
                    label = class_labels[pred]
                    
                    angry_acc = round(preds[0][0] * 100, 1)
                    contempt_acc = round(preds[0][1] * 100, 1)
                    disgust_acc = round(preds[0][2] * 100, 1)
                    fear_acc = round(preds[0][3] * 100, 1)
                    happy_acc = round(preds[0][4] * 100, 1)
                    neutral_acc = round(preds[0][5] * 100, 1)
                    sad_acc = round(preds[0][6] * 100, 1)
                    surprise_acc = round(preds[0][7] * 100, 1)
                    pred_acc = round(preds[0][pred] * 100, 1)
                    
                    pred_summary = f"angry: {angry_acc}\ncontempt: {contempt_acc}\ndisgust: {disgust_acc}\nfear: {fear_acc}\nhappy: {happy_acc}\nneutral: {neutral_acc}\nsad: {sad_acc}\nsurprise: {surprise_acc}\n"
                    pred_summary_path = os.path.join(os.path.join(valid_cropped_split_path, sd))
                    pred_summary_path += f'\img_{i}'
                    pred_summary_file = open(f'{pred_summary_path}.txt', 'w')
                    
                    cv2.rectangle(img, (face_x, face_y), (face_x + face_w, face_y + face_h), (0, 255, 0), 1)
                    cv2.putText(img, f'{label}: {pred_acc}%', (face_x, face_y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.55, (0, 255, 0), 1)
                    
                    cv2.imwrite(os.path.join(f'{pred_summary_path}.jpg'), img)
                    pred_summary_file.write(pred_summary)
                    pred_summary.close()
        except:
            corrupted += 1

corrupted

347

In [13]:
valid_X = []
valid_y = []
sub_dirs = os.listdir(valid_split_path)
corrupted = 0

for sd in sub_dirs:
    sd_path = os.path.join(valid_split_path, sd)
    img_files = os.listdir(sd_path)

    for i, f in enumerate(img_files):
        img_file_path = os.path.join(sd_path, f)

        try:
            img = cv2.imread(img_file_path)
            h, w, _ = img.shape
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            face_detect_results = face_detector.process(img_rgb)
            img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
            
            if face_detect_results.detections:
                for face in face_detect_results.detections:
                    rel_bound_rect = face.location_data.relative_bounding_box
                    face_x, face_w, face_y, face_h = int(rel_bound_rect.xmin * w), int(rel_bound_rect.width * w), int(rel_bound_rect.ymin * h), int(rel_bound_rect.height * h)
                    cropped_face = img_gray[face_y: face_y + face_h, face_x: face_x + face_w]
                    processed_img = cv2.resize(cropped_face, (48, 48))
                    valid_X.append(processed_img)
                    valid_y.append(label_classes[sd])
        except:
            corrupted += 1

valid_X = np.array(valid_X)
valid_y = np.array(valid_y)
valid_X = valid_X / 255.0
valid_X = np.reshape(valid_X, (valid_X.shape[0], valid_X.shape[1], valid_X.shape[2], 1))
valid_y = np_utils.to_categorical(valid_y)

print(valid_X.shape, valid_y.shape)

(339, 48, 48, 1) (339, 8)


In [15]:
model.load_weights('../models/VGG16-best_loss_model.h5')

In [16]:
loss, acc = model.evaluate(valid_X, valid_y)

print(loss, acc)

11/11 [==============================] - 6s 550ms/step - loss: 4.1993 - accuracy: 0.5251
4.19926118850708 0.525073766708374


In [17]:
model.load_weights('../models/VGG16-best_val_loss_model.h5')

In [18]:
loss, acc = model.evaluate(valid_X, valid_y)

print(loss, acc)

11/11 [==============================] - 5s 465ms/step - loss: 3.1193 - accuracy: 0.5162
3.11932110786438 0.516224205493927
